# Exploratory Data Analysis: King County Housing

In [ ]:
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from matplotlib.ticker import PercentFormatter

In [ ]:
# configure default figure size and colors for matplotlib plots
plt.rcParams.update({ "figure.figsize" : (8, 5),"axes.facecolor" : "white", "axes.edgecolor":  "black"})
# set figure background color to white
plt.rcParams["figure.facecolor"]= "w"
# register pandas converters for handling datetime data in matplotlib
pd.plotting.register_matplotlib_converters()
# set pandas display option to format floating-point numbers to 3 decimal places
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# seaborn whitegrid style for plots
sns.set_style("whitegrid")

In [ ]:
df = pd.read_csv('data/prices.csv')
df2 = pd.read_csv('data/sales.csv')
df2.head()

In [ ]:
# merge sales and prices data, drop and rename columns
merged_df = pd.merge(df, df2, left_on='id', right_on='house_id', how='inner')
merged_df = merged_df.drop(columns=['house_id'])
merged_df = merged_df.rename(columns={'id_x': 'id', 'id_y': 'sales_id'})
merged_df.head()

In [ ]:
from geopy.geocoders import Nominatim
from tqdm import tqdm

# initialize Nominatim geocoder
geolocator = Nominatim(user_agent="housing_prices_app")

# function to reverse geocode coordinates to address
def reverse_geocode(lat, long):
    location = geolocator.reverse((lat, long))
    return location.address if location else None

# apply reverse geocoding to each row in the DataFrame
tqdm.pandas()  # view progress bar
merged_df['address'] = df.progress_apply(lambda row: reverse_geocode(row['lat'], row['long']), axis=1)
merged_df.to_csv('combined_data_with_address.csv', index=False)

In [ ]:
merged_df = pd.read_csv('data/combined_data_with_address.csv')
merged_df.head()

In [ ]:
print(merged_df.info())

In [ ]:
# add a colum with price per square meter

merged_df['price_per_sqm'] = merged_df.price/(merged_df.sqft_living/10.764)
merged_df.head()

In [ ]:
# function to format the y-axis labels
def format_price_in_100k(x, pos):
    return f'{x/100000:.0f}'

In [ ]:
# check if there are duplicates in the id column
merged_df.duplicated('id').sum()

In [ ]:
# create another df with only only one price per house (latest price)

# sort df by id and date
merged_df_sorted = merged_df.sort_values(by=['id', 'date'])

# keep only the last occurrence of each id
df_latest = merged_df_sorted.drop_duplicates(subset='id', keep='last')

# reset index
df_latest.reset_index(drop=True, inplace=True)

# check if there are duplicates in the id column
print(df_latest.duplicated('id').sum())
print(df_latest.info())

In [ ]:
# boxplot for waterfront yes or no

# replace 0, 1 in waterfront with no, yes
merged_df['waterfront'] = merged_df['waterfront'].replace({0: 'No', 1: 'Yes'})

df_latest['waterfront'] = df_latest['waterfront'].replace({0: 'No', 1: 'Yes'})
df_latest.head()

In [ ]:
# set up the boxplot
plt.figure(figsize=(8, 4.5))
sns.boxplot(x='waterfront', y='price_per_sqm', data=df_latest)

# apply the format function to the y-axis
# plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(format_price_in_100k))

# set labels
plt.xlabel('Waterfront')
plt.ylabel('Price per square meter')
plt.title('House Prices by Waterfront')

plt.show()

In [ ]:
# new grouped df for waterfront plot
df_waterfront = df_latest.groupby('waterfront')['price_per_sqm'].mean().reset_index()
print(df_waterfront.columns)

# create barplot
plt.figure(figsize=(8, 4.5))
sns.barplot(x='waterfront', y='price_per_sqm', data=df_waterfront)

# format y-axis and set ticks to 500 intervals
# plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(format_price_in_100k))
plt.gca().yaxis.set_major_locator(plt.MultipleLocator(500))

# set labels
plt.xlabel('Waterfront')
plt.ylabel('Price per square meter')
plt.title('Mean House Prices by Waterfront')

plt.ylim(0, 6000)

plt.show()

In [ ]:
# plot with price by number of bedrooms

# drop row with house that has 33 bedrooms (probably data entry error)
df_latest = df_latest.drop(df_latest[df_latest['bedrooms'] == 33].index)

# group data by mean, min, max
mean_prices = df_latest.groupby('bedrooms')['price_per_sqm'].mean().reset_index()
min_prices = df_latest.groupby('bedrooms')['price_per_sqm'].min().reset_index()

# concatenate the two dfs
prices = pd.concat([mean_prices, min_prices, median_prices], keys=['mean', 'min']).reset_index()

# create barplot
plt.figure(figsize=(16, 9))
sns.barplot(data=prices, x=prices['bedrooms'].astype(int), y='price_per_sqm', hue='level_0', palette=['blue', 'skyblue']) # hue maps differnt colors to the two bars

# set labels
plt.xlabel('Number of Bedrooms')
plt.ylabel('Price per square meter')
plt.title('House Prices by Number of Bedrooms')

# set ticks at intervals of 500 on y-axis
# plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(format_price_in_100k))
plt.gca().yaxis.set_major_locator(plt.MultipleLocator(500))

# add legend and set color
legend = plt.legend(labels=['Mean price', 'Minimum price'], loc='upper right')
legend.legendHandles[0].set_color('blue')
legend.legendHandles[1].set_color('skyblue')

# set y-axis limit to 6000
plt.ylim(0, 4500)

plt.show()

In [ ]:
# group data by grade
grouped_data = df_latest.groupby('grade')['price_per_sqm'].agg(['mean', 'min']).reset_index()

# rename columns
grouped_data.columns = ['grade', 'mean_price', 'min_price']

# new df with just grade, mean_price, and median_price
df_grade = grouped_data[['grade', 'mean_price', 'min_price']]

# reshape the df to have 'mean' and 'min' in separate rows
melted_df = df_grade.melt(id_vars='grade', var_name='stat', value_name='price_per_sqm')

# create barplot
plt.figure(figsize=(16, 9))
sns.barplot(data=melted_df, x='grade', y='price_per_sqm', hue='stat', palette={'mean_price': 'blue', 'min_price': 'skyblue'})

# set labels
plt.xlabel('House Grade')
plt.ylabel('Price per square meter')
plt.title('Mean and Minimum Prices by House Grade')

# set labels and colors
legend = plt.legend(labels=['Mean price', 'Minimum price'])
legend.legendHandles[0].set_color('blue')
legend.legendHandles[1].set_color('skyblue')

# set ticks at intervals of 500 on y-axis
# plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(format_price_in_100k))
plt.gca().yaxis.set_major_locator(plt.MultipleLocator(500))

# set y-axis limit to 1.5 million
plt.ylim(0, 5500)

plt.show()

In [ ]:
merged_df.zipcode.unique()

In [ ]:
# dictionary to assign one of 6 broader areas to each zip code
zipcode_to_area = ({
    98002: 'South King County',
    98155: 'North King County',
    98188: 'Southwest King County',
    98059: 'Southwest King County',
    98118: 'Seattle',
    98003: 'South King County',
    98023: 'South King County',
    98117: 'Seattle',
    98034: 'North King County',
    98022: 'East Central King County',
    98116: 'Seattle',
    98001: 'South King County',
    98010: 'East Central King County',
    98198: 'Southwest King County',
    98042: 'South King County',
    98038: 'South King County',
    98027: 'Eastside',
    98045: 'East Central King County',
    98144: 'Seattle',
    98029: 'Eastside',
    98075: 'Eastside',
    98199: 'Seattle',
    98107: 'Seattle',
    98105: 'Seattle',
    98052: 'Eastside',
    98053: 'Eastside',
    98014: 'East Central King County',
    98077: 'Eastside',
    98115: 'Seattle',
    98011: 'Eastside',
    98008: 'Eastside',
    98019: 'Eastside',
    98092: 'Eastside',
    98074: 'Eastside',
    98030: 'South King County',
    98103: 'Seattle',
    98004: 'Eastside',
    98126: 'Seattle',
    98177: 'Seattle',
    98056: 'North King County',
    98133: 'Seattle',
    98112: 'Seattle',
    98033: 'Eastside',
    98028: 'North King County',
    98007: 'Eastside',
    98005: 'Eastside',
    98136: 'Seattle',
    98031: 'South King County',
    98032: 'South King County',
    98168: 'Southwest King County',
    98006: 'Eastside',
    98178: 'Southwest King County',
    98072: 'North King County',
    98102: 'Seattle',
    98125: 'Seattle',
    98146: 'Seattle',
    98166: 'Seattle',
    98122: 'Seattle',
    98070: 'Seattle',
    98148: 'Southwest King County',
    98040: 'Eastside',
    98106: 'Seattle',
    98065: 'East Central King County',
    98108: 'Seattle',
    98058: 'Southwest King County',
    98024: 'East Central King County',
    98055: 'South King County',
    98109: 'Seattle',
    98119: 'Seattle',
    98039: 'Eastside'
})

# map zip codes to areas and create new column 'area' in merged_df
merged_df['area'] = merged_df['zipcode'].map(zipcode_to_area)

df_latest['area'] = df_latest['zipcode'].map(zipcode_to_area)
df_latest.head()

In [ ]:
# check if every row as an area
print(merged_df.isnull().sum())
print(df_latest.isnull().sum())

In [ ]:
# group data by grade
grouped_data2 = df_latest.groupby('area')['price_per_sqm'].agg(['mean', 'min']).reset_index()

# rename columns
grouped_data2.columns = ['area', 'mean_price', 'min_price']

# new df with just area, mean_price, and median_price
df_area = grouped_data2[['area', 'mean_price', 'min_price']]

# reshape the df to have 'mean' and 'min' in separate rows
melted_df2 = df_area.melt(id_vars='area', var_name='stat', value_name='price_per_sqm')

# create barplot
plt.figure(figsize=(16, 9))
sns.barplot(data=melted_df2, x='area', y='price_per_sqm', hue='stat', palette={'mean_price': 'blue', 'min_price': 'skyblue'})

# set labels and title
plt.xlabel('King County Area')
plt.ylabel('Price per square meter')
plt.title('Mean and Minimum Prices by Area')

# set legend title and labels
legend = plt.legend(labels=['Mean price', 'Minimum price'])
legend.legendHandles[0].set_color('blue')
legend.legendHandles[1].set_color('skyblue')

# set ticks at intervals of 500 on the y-axis
# plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(format_price_in_100k))
plt.gca().yaxis.set_major_locator(plt.MultipleLocator(500))

plt.ylim(0,4000)

plt.show()

In [ ]:
# check the dtpye of date column
merged_df.date.info()

In [ ]:
# convert date column to datetime
merged_df['date'] = pd.to_datetime(merged_df['date'])

# sort df by date
merged_df = merged_df.sort_values(by='date')
merged_df.date.info()

In [ ]:
from matplotlib.dates import MonthLocator, DateFormatter

# show min and max of date column
print("Minimum Date:", merged_df.date.min())
print("Maximum Date:", merged_df.date.max())

# calculate mean, min, and max prices for each date
mean_prices = merged_df.groupby('date')['price_per_sqm'].mean().reset_index()
min_prices = merged_df.groupby('date')['price_per_sqm'].min().reset_index()
max_prices = merged_df.groupby('date')['price_per_sqm'].max().reset_index()

# create lineplots
plt.figure(figsize=(16, 4.5))
sns.lineplot(data=mean_prices, x='date', y='price_per_sqm', label='Mean price')
sns.lineplot(data=min_prices, x='date', y='price_per_sqm', label='Minimum price')
sns.lineplot(data=max_prices, x='date', y='price_per_sqm', label='Maximum price')

# set labels and title
plt.title('House Prices Over Time')
plt.xlabel('Date')
plt.ylabel('Price per square meter')

# set custom price format and ticks at intervals of 200k on the y-axis
# plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(format_price_in_100k))
plt.gca().yaxis.set_major_locator(plt.MultipleLocator(1000))

# set x-axis ticks to one month intervals
plt.gca().xaxis.set_major_locator(MonthLocator())
plt.gca().xaxis.set_major_formatter(DateFormatter('%Y-%m'))

# add grid lines
plt.grid(True)
# fit the plot
plt.tight_layout()

plt.ylim(0,11000)

plt.show()

In [ ]:
# create lineplot
plt.figure(figsize=(16, 4.5))
sns.lineplot(data=merged_df, x='date', y='price_per_sqm', marker='o', ci=None) # disable confidence interval

# set labels and title
plt.title('Mean House Price Over Time')
plt.xlabel('Date')
plt.ylabel('Price per square meter')

# set custom price format and ticks at intervals of 200k on the y-axis
# plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(format_price_in_100k))
plt.gca().yaxis.set_major_locator(plt.MultipleLocator(1000))

# set x-axis ticks to one month intervals
plt.gca().xaxis.set_major_locator(MonthLocator())
plt.gca().xaxis.set_major_formatter(DateFormatter('%Y-%m'))

# add grid lines
plt.grid(True)
# fit the plot
plt.tight_layout()

plt.show()

In [ ]:
print('The smallest living area is', round(merged_df.sqft_living.min()), 'square feet, which is', round(merged_df.sqft_living.min()/10.764), 'square meters')
print('The largest living area is', round(merged_df.sqft_living.max()), 'square feet, which is', round(merged_df.sqft_living.max()/10.764), 'square meters')

In [ ]:
# create bin edges with smaller ranges at the beginning and bigger at the end
bin_edges = [30, 70, 110, 150, 190, 230, 270, 500, 750, 1000, 1250]

# create custom bin labels
bin_labels = [f'{int(bin_edges[i])} - {int(bin_edges[i+1])}' for i in range(len(bin_edges)-1)]

# create equally sized bins for the sqft_living column
df_latest['sqm_living_bins'] = pd.cut((df_latest['sqft_living']/10.764), bins=bin_edges, labels=bin_labels)

# group data by bins and calculate mean price for each bin
prices_by_bin = df_latest.groupby('sqm_living_bins')['price_per_sqm'].agg(['mean', 'min']).reset_index()

# reshape the df to have 'mean' and 'min' in separate rows
melted_df3 = pd.melt(prices_by_bin, id_vars='sqm_living_bins', var_name='stat', value_name='price_per_sqm')

# create bar plot
plt.figure(figsize=(16, 9))
sns.barplot(data=melted_df3, x='sqm_living_bins', y='price_per_sqm', hue='stat', palette={'mean': 'blue', 'min': 'skyblue'})

# set labels and title
plt.xlabel('Living Area Ranges (in square meters)')
plt.ylabel('Price per square meter')
plt.title('Mean House Prices by Living Area')

# set custom price format and ticks at intervals of 200k on the y-axis
# plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(format_price_in_100k))
plt.gca().yaxis.set_major_locator(plt.MultipleLocator(500))

# limit y-axis
# plt.ylim(0,2500000)

# set legend title and labels
legend = plt.legend(labels=['Mean price', 'Minimum price'])
legend.legendHandles[0].set_color('blue')
legend.legendHandles[1].set_color('skyblue')

plt.show()

In [ ]:
# create bin edges with smaller ranges at the beginning and bigger at the end
bin_edges = [30, 70, 110, 150, 190, 230, 270, 500, 750, 1000, 1250]

# create custom bin labels
bin_labels = [f'{int(bin_edges[i])} - {int(bin_edges[i+1])}' for i in range(len(bin_edges)-1)]

# create equally sized bins for the sqft_living column
df_latest['sqm_living_bins'] = pd.cut((df_latest['sqft_living']/10.764), bins=bin_edges, labels=bin_labels)

# group data by bins and calculate mean price for each bin
prices_by_bin = df_latest.groupby('sqm_living_bins')['price'].agg(['mean', 'min']).reset_index()

# reshape the df to have 'mean' and 'min' in separate rows
melted_df3 = pd.melt(prices_by_bin, id_vars='sqm_living_bins', var_name='stat', value_name='price')

# create bar plot
plt.figure(figsize=(16, 9))
sns.barplot(data=melted_df3, x='sqm_living_bins', y='price', hue='stat', palette={'mean': 'blue', 'min': 'skyblue'})

# set labels and title
plt.xlabel('Living Area Ranges (in square meters)')
plt.ylabel('Price (in 100k)')
plt.title('Mean House Prices by Living Area')

# set custom price format and ticks at intervals of 200k on the y-axis
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(format_price_in_100k))
plt.gca().yaxis.set_major_locator(plt.MultipleLocator(200000))

# limit y-axis
plt.ylim(0,2500000)

# set legend title and labels
legend = plt.legend(labels=['Mean price', 'Minimum price'])
legend.legendHandles[0].set_color('blue')
legend.legendHandles[1].set_color('skyblue')

plt.show()

In [ ]:
# check why mean is so high at the end of May
merged_df[merged_df['date'] == '2015-05-27'] #only one availabe house at 1.31m

In [ ]:
# check which house has 33 bedrooms (doesn't look like a 33 bedroom house on Google Street View)
df_latest[df_latest['bedrooms'] == 33]

In [ ]:
# check which houses have grade 3
df_latest[df_latest['grade'] == 3]

In [ ]:
# check which houses have 8 bedrooms
houses_with_8_bedrooms = df_latest[df_latest['bedrooms'] == 8]

# find row with the highest price among houses with 8 bedrooms
house_with_highest_price = houses_with_8_bedrooms[houses_with_8_bedrooms['price'] == houses_with_8_bedrooms['price'].max()]
print(house_with_highest_price.price)
houses_with_8_bedrooms

In [ ]:
# look for houses that fit buyer's criteria

# define conditions
condition_grade = (df_latest['grade'].between(4, 8))
condition_area = (df_latest['area'].isin(['South King County', 'Southwest King County']))
condition_bedrooms = (df_latest['bedrooms'].between(5, 6))
condition_price = (df_latest['price'] < 200000)

# apply conditions to df
filtered_df = df_latest[condition_grade & condition_area & condition_bedrooms & condition_price]

filtered_df

In [ ]:
df_latest.describe()